In [179]:
import yfinance as yf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.dates as mdates

In [180]:
def fetch_data(ticker, start_date, end_date):
    return yf.download(ticker, start=start_date, end=end_date)

In [181]:
def calculate_ma(data, window=20):
    data['MA'] = data['Close'].rolling(window=window).mean()
    return data

In [182]:
def calculate_ema(data, span=20):
    data['EMA'] = data['Close'].ewm(span=span, adjust=False).mean()
    return data

In [183]:
def calculate_rsi(data, window=14):
    delta = data['Close'].diff(1)
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=window).mean()
    avg_loss = loss.rolling(window=window).mean()
    rs = avg_gain / avg_loss
    data['RSI'] = 100 - (100 / (1 + rs))
    return data

In [184]:
def calculate_macd(data, span_short=12, span_long=26, span_signal=9):
    data['MACD'] = data['Close'].ewm(span=span_short, adjust=False).mean() - data['Close'].ewm(span=span_long, adjust=False).mean()
    data['Signal Line'] = data['MACD'].ewm(span=span_signal, adjust=False).mean()
    return data

In [185]:
def calculate_bollinger_bands(data, window=20):
    data['Rolling_STD'] = data['Close'].rolling(window=window).std()
    data['Upper Band'] = data['MA'] + (2 * data['Rolling_STD'])
    data['Lower Band'] = data['MA'] - (2 * data['Rolling_STD'])
    return data

In [186]:
def calculate_stochastic_oscillator(data, window=14):
    low_min = data['Low'].rolling(window=window).min()
    high_max = data['High'].rolling(window=window).max()
    data['%K'] = (data['Close'] - low_min) * 100 / (high_max - low_min)
    data['%D'] = data['%K'].rolling(window=3).mean()
    return data

In [187]:
def calculate_atr(data, window=14):
    high_low = data['High'] - data['Low']
    high_close = np.abs(data['High'] - data['Close'].shift(1))
    low_close = np.abs(data['Low'] - data['Close'].shift(1))
    tr = np.maximum(high_low, high_close)
    tr = np.maximum(tr, low_close)
    data['ATR'] = tr.rolling(window=window).mean()
    return data

In [188]:
# On-Balance Volume
def calculate_obv(data):
    obv = [0]  # Starting OBV value
    for i in range(1, len(data)):
        if data['Close'][i] > data['Close'][i - 1]:
            obv.append(obv[-1] + data['Volume'][i])
        elif data['Close'][i] < data['Close'][i - 1]:
            obv.append(obv[-1] - data['Volume'][i])
        else:
            obv.append(obv[-1])
    data['OBV'] = obv
    return data

In [189]:
def calculate_obv_guard(data):
    obv = [0]  # Starting OBV value
    for i in range(1, len(data)):
        current_close = data['Close'].iloc[i]
        previous_close = data['Close'].iloc[i - 1]
        current_volume = data['Volume'].iloc[i]

        if current_close > previous_close:
            obv.append(obv[-1] + current_volume)
        elif current_close < previous_close:
            obv.append(obv[-1] - current_volume)
        else:
            obv.append(obv[-1])
    data['OBV'] = pd.Series(obv, index=data.index)  # Ensure OBV aligns with the DataFrame index
    return data

In [190]:
def plot_indicators(data, ticker):
    plt.figure(figsize=(14, 20))

    # Plot Closing Price and Moving Averages
    plt.subplot(8, 1, 1)
    plt.plot(data['Close'], label='Close Price')
    plt.plot(data['MA'], label='MA (20)')
    plt.plot(data['EMA'], label='EMA (20)')
    plt.title(f'{ticker} Stock Price and Moving Averages')
    plt.legend()

    # Plot RSI
    plt.subplot(8, 1, 2)
    plt.plot(data['RSI'], label='RSI (14)')
    plt.axhline(y=70, color='r', linestyle='--')
    plt.axhline(y=30, color='r', linestyle='--')
    plt.title('RSI')
    plt.legend()

    # Plot MACD
    plt.subplot(8, 1, 3)
    plt.plot(data['MACD'], label='MACD (12, 26)')
    plt.plot(data['Signal Line'], label='Signal Line (9)')
    plt.title('MACD')
    plt.legend()

    # Plot Bollinger Bands
    plt.subplot(8, 1, 4)
    plt.plot(data['Close'], label='Close Price')
    plt.plot(data['Upper Band'], label='Upper Band')
    plt.plot(data['Lower Band'], label='Lower Band')
    plt.title('Bollinger Bands')
    plt.legend()

    # Plot Stochastic Oscillator
    plt.subplot(8, 1, 5)
    plt.plot(data['%K'], label='%K')
    plt.plot(data['%D'], label='%D')
    plt.axhline(y=80, color='r', linestyle='--')
    plt.axhline(y=20, color='r', linestyle='--')
    plt.title('Stochastic Oscillator')
    plt.legend()

    # Plot ATR
    plt.subplot(8, 1, 6)
    plt.plot(data['ATR'], label='ATR (14)')
    plt.title('Average True Range (ATR)')
    plt.legend()

    # Plot OBV
    # plt.subplot(8, 1, 7)
    # plt.plot(data['OBV'], label='OBV')
    # plt.title('On-Balance Volume (OBV)')
    # plt.legend()

    # Plot Volume 
    plt.subplot(8, 1, 8) 
    # data['Volume'].plot(kind='bar', label='Volume', width=1.0) 
    # plt.bar(data.index, data['Volume'], label='Volume', width=1.0)
    # normalized_volume = data['Volume'] / data['Volume'].max()
    # plt.bar(x=mdates.date2num(data.index), height=data['Volume'].max(), data=normalized_volume, label='Volume')
    # plt.title('Volume')
    plt.plot(data['Volume'], label='Volume')
    plt.title('Volume')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

In [191]:
# Main function to process multiple tickers
def process_tickers(tickers, start_date, end_date):
    for ticker in tickers:
        data = fetch_data(ticker, start_date, end_date)
        data = calculate_ma(data)
        data = calculate_ema(data)
        data = calculate_rsi(data)
        data = calculate_macd(data)
        data = calculate_bollinger_bands(data)
        data = calculate_stochastic_oscillator(data)
        data = calculate_atr(data)
        # data = calculate_obv_guard(data)
        plot_indicators(data, ticker)

In [192]:
# Example usage
tickers = ['ROK','NVDA'] 
start_date = '2022-01-01'
end_date = '2024-12-31'
process_tickers(tickers, start_date, end_date)


[*********************100%***********************]  1 of 1 completed


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().